In [1]:
import math

import numpy as np

import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import gridspec

from scipy.interpolate import InterpolatedUnivariateSpline

from matplotlib import figure
from ipywidgets import interact

In [2]:
SNeff_path = 'MDet_yrs_vs_SNeff.csv'
SNeff_file = open(SNeff_path, "r")

MDet_years1 = []
SNeffs1 = []

lines = SNeff_file.readlines()
for line in lines[6:36]:
    vals = line.split(',')
    MDet_years1.append(float(vals[2]))
    SNeffs1.append(float(vals[3]))
    

MDet_to_SNeff = InterpolatedUnivariateSpline(MDet_years1, SNeffs1)

In [3]:
Freeze_path = 'Freeze_outs.csv'
Freeze_file = open(Freeze_path, "r")

SNeffs2 = []
spin0 = []
spin05 = []
spin1 = []

lines = Freeze_file.readlines()
for line in lines[10:]:
    vals = line.split(',')
    SNeffs2.append(float(vals[0]))
    
    spin0.append(float(vals[1]))
    spin05.append(float(vals[2]) if len(vals[2]) > 0 else math.nan)
    spin1.append(float(vals[3]) if len(vals[3]) > 0 else math.nan)

SNeff_to_spin0 = InterpolatedUnivariateSpline(SNeffs2, spin0)
SNeff_to_spin05 = InterpolatedUnivariateSpline(SNeffs2[20:], spin05[20:])
SNeff_to_spin1 = InterpolatedUnivariateSpline(SNeffs2[27:], spin1[27:])

In [8]:
@interact
def my_widget(SO_Used     = True , SO_Start     = (2024, 2028), 
              AdvSO_Used  = True , AdvSO_Start  = (2029, 2035), AdvSO_Duration  = (7,10),
              CHLAT1_Used = True , CHLAT1_Start = (2030, 2035), CHLAT1_Duration = (10,20),
              CHALT2_Used = False, CHLAT2_Start = (2030, 2035), CHLAT2_Duration = (10,20)   ):
    
    Detectors    = [0.031, 0.062  , 0.135    , 0.135    ] 
    Efficiencies = [0.9  , 0.9    , 1.0      , 1.0      ]
    Names        = ["SO" , "AdvSO", "CHLAT 1", "CHLAT 2"]
    
    StartYears   = [SO_Start, AdvSO_Start,  CHLAT1_Start,  CHLAT2_Start]
    Durations    = [AdvSO_Start-SO_Start,  AdvSO_Duration,  CHLAT1_Duration,  CHLAT2_Duration]
    Used         = [SO_Used,  AdvSO_Used,  CHLAT1_Used,  CHALT2_Used]

    FirstYear = np.min(StartYears)
    LastYear = np.max(np.add(StartYears,Durations))

    increment = 1/365

    years = np.arange(2024, 2037, increment)
    dates = pd.date_range(start = '2024-01-01', end = '2037-01-01', freq = 'D')
    det_times = []
    total_det_time = 0

    for date in years:
        for i in range(len(Detectors)):
            if Used[i]:
                if date >= StartYears[i] and date <= StartYears[i] + Durations[i]:
                    total_det_time += Detectors[i]*Efficiencies[i]*increment

        det_times.append(total_det_time)

    These_SNeffs = MDet_to_SNeff(det_times)

    These_spin0s = SNeff_to_spin0(These_SNeffs)
    These_spin05s = SNeff_to_spin05(These_SNeffs)
    These_spin1s = SNeff_to_spin1(These_SNeffs)


    #===============================================================================================


    %matplotlib inline

    fig = plt.figure(figsize=(20, 10), dpi=100, constrained_layout=False)
    #fig.suptitle('Time-series Comparason', fontsize=24)
    gs1 = fig.add_gridspec(nrows = 1, ncols = 1, wspace=0.001) 

    ax0 = plt.subplot(gs1[0, 0])

    ax0.plot(years[:], These_spin0s[:], label='Spin 0')
    ax0.plot(years[:], These_spin05s[:], label='Spin 1/2')
    ax0.plot(years[:], These_spin1s[:], label='Spin 1')

    for i in range(len(StartYears)):
        ax0.plot((StartYears[i], StartYears[i]), (100, 10000), scaley = False, c = "#878787")
        ax0.text(StartYears[i] - 0.25, 12000, Names[i], rotation = 90, fontsize = 23, c = "#878787")

    ax0.set_yscale('log')

    ax0.set_xlabel('Year', fontsize = 30)
    ax0.set_ylabel('Freeze Out Temp [MeV]', fontsize = 30)
    ax0.tick_params(axis='x', labelsize=23)
    ax0.tick_params(axis='y', labelsize=23)

    ax0.legend()

    # fig.tight_layout()
    fig.savefig("Test-Timeline-Graph.png")
    plt.show()

interactive(children=(Checkbox(value=True, description='SO_Used'), IntSlider(value=2026, description='SO_Start…